In [1]:
import requests
import json
import os
import datetime
import hashlib
from flask import Flask, jsonify, request
from uuid import uuid4
from urllib.parse import urlparse

In [2]:
class PlutoChain:
    def __init__(self):
        self.chain = []
        self.transaction = []
        self.addBlock(proof = 1 , prevHash ="0")
        self.nodes = set()
        
    def addBlock(self, proof, prevHash = "0"):
        block = {
            "index": len(self.chain),
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevHash": prevHash,
            "transactions": self.transaction
        }
        self.transaction = []
        self.chain.append(block)
        return block
    
    def getLastBlock(self):
        return self.chain[-1]
    
    def proofOfWork(self, prevProof):
        new_proof = 1
        solved = False
        
        while solved is False:
            hashWork = hashlib.sha256(str(new_proof ** 2 - prevProof ** 2).encode()).hexdigest()
            if hashWork[:4] == "0000":
                solved = True
            else:
                new_proof +=1 
        return new_proof
    
    def hashBlock(self, block):
        encodedBlock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encodedBlock).hexdigest()
    
    def isChainValid(self, chain):
        prevBlock = chain[0]
        current_index = 1
        
        while current_index < len(chain):
            current_block = chain[current_index]
            current_block_prevHash = current_block["prevHash"]
            if current_block_prevHash != self.hashBlock(prevBlock):
                return False
            currentProof = current_block["proof"]
            prevProof = prevBlock["proof"]
            hashWork = hashlib.sha256(str(currentProof ** 2 - prevProof ** 2).encode()).hexdigest()
            if hashWork[:4] != "0000":
                return False
            prevBlock = current_block
            current_index += 1
        return True
    
    def addTransaction(self, sender, receiver, amount):
        self.transaction.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })
        previousBlock = self.getLastBlock()
        return previousBlock["index"] + 1
    
    def addNode(self, address):
        nodeIP = urlparse(address)
        self.nodes.add(nodeIP.netloc)
    
    ### making sure that the blocks are all the same length
    ### each node wull contain a version of the chain 
    ## if not the replace will have to occur in the node
    def replaceChain(self):
        ## all the cmputer all arond the world
        network = self.nodes
        ##cinsideration of the longet chain 
        longestChain = None
        ## for loop though the entire the chain and find the longest chain and assign it 
        ### if we find a chain that is longer than the chain we are on we will updae the 
        ##Max length var will then be updates
        max_Length = len(self.chain)
        for node in network:
            ##request to get the chain using the revious function weve made
            response = requests.get(f"http://{node}/chain")
            ## Check if everying is working 
            if response.status_code == 200:
                length = response.json()["length"]
                chain = response.json()["chain"]
                if length > max_Length and self.isChainValid(chain):
                    max_Length = length
                    longestChain = chain
            if longestChain:
                self.chain = longestChain
                return True
            ### Chain was not replaced
            return False

In [3]:
nodeAddress = str(uuid4()).replace('-', '')
### needed to keep track of the mining address and the award received for a successful mine
### Transaction from the node to the miner 
## uuid4 library generate a uniqie address for out node

In [4]:
pluto = PlutoChain()

In [5]:
app = Flask(__name__)

In [6]:
## need to post the transactions into a json file before we can call it 
@app.route("/addTransaction", methods = ["POST"])
def addTransaction():
    ### function has to create a transaciton 
    ### need sender, receiver, amount 
    json = request.get_json()
    ## make sure al lthe keys are present in the json file 
    ### Transactions have to consist of all of these keys 
    transactionKeys = ["sender", "receiver", "amount"]
    if not all (key in json for key in transactionKeys): 
        return "Some elements of transaction is missing", 400
    ### add transaciton to the next mined block
    index = pluto.addTransaction(json["sender"], json["receiver"], json["amount"])
    response = {
        "success": f"transaction will be added to index {index}"
    }
    return jsonify(response), 201

In [7]:
##connecting new nodes the nodes -> decentralizing the crypto "POST" -> Creating a new node -> need to register node
@app.route("/connectNode", methods = ["POST"])
def connectNode():
    nodeJson = request.get_json()
    ##connect a new node to all the other nodes
    ## need to get the address we needfor the addNode function 
    ### Will contain all the nodes in the network
    nodes = nodeJson.get('nodes')
    ## check nodes are not empty 
    if nodes is None:
        return "No Node", 400
    for node in nodes:
        pluto.addNode(node)
    response = {
        "message": "All nodes are now collected",
        "totalnodes": list(pluto.nodes)
    }
    return jsonify(response), 201


In [8]:
### Updateing the chain/ replacing with the longest 
@app.route("/replacechain", methods = ["GET"])
def replacechain():
    ##get boolean true or false is the chain needs to me replaced
    shouldreplace = pluto.replaceChain()
    if shouldreplace:
        response = {
            "message": "The chain was replaced",
            "newchain": pluto.chain,
            "chainlength": len(pluto.chain)
        }
    else:
        response = {
            "message": "All good chain is at the largest",
            "currentchain": pluto.chain,
            "chainlength": len(pluto.chain)
        }
    return jsonify(response), 200

In [9]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousBlock = pluto.getLastBlock()
    previousProof = previousBlock["proof"]
    proof = pluto.proofOfWork(previousProof)
    prevHash = pluto.hashBlock(previousBlock)
    pluto.addTransaction(sender= nodeAddress, receiver= "Me", amount= 10)
    block = pluto.addBlock(proof, prevHash)
    response = {
        "index": block["index"],
        "stamp": block["timestamp"],
        "proof": block["proof"],
        "transactions": block["transactions"],
        "prevHash": block["prevHash"]
    }
    return jsonify(response), 200

In [10]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": pluto.chain,
        "length": len(pluto.chain)
    }
    return jsonify(response),200

In [11]:
app.run(host="0.0.0.0", port=5001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Dec/2021 22:46:02] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:46:52] "POST /connectNode HTTP/1.1" 201 -
127.0.0.1 - - [29/Dec/2021 22:47:28] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:47:31] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:47:35] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:48:17] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:54:59] "POST /addTransaction HTTP/1.1" 201 -
127.0.0.1 - - [29/Dec/2021 22:55:35] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:56:10] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:57:10] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 22:59:41] "POST /addTransaction HTTP/1.1" 201 -
127.0.0.1 - - [29/Dec/2021 23:00:01] "POST /addTransaction HTTP/1.1" 201 -
127.0.0.1 - - [29/Dec/2021 23:00:14] "POST /addTransaction HTTP/1.1" 201 -
127.0.0.1 - - [29/Dec/2021 23:00:21] "POST /mine HTTP